# 2. Dataset Setup & Exploration
**Objective:** Download and inspect the Intel Image Classification dataset

In [1]:
#Set up necessary installs
%pip install tensorflow
!pip install tensorflow
!pip install kaggle
!pip install matplotlib
!pip install numpy
!pip install pillow
import glob
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import Model,layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

  Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.5.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.17.2-cp312-cp312-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.4 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached h5py-3.13.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using 

2025-03-07 16:48:25.981523: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-07 16:48:25.985782: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-07 16:48:25.997757: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741366106.018758     425 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741366106.024879     425 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 16:48:26.045627: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [7]:
import os

# Get the current working directory
current_directory = os.getcwd()

print("Current working directory:", current_directory)

Current working directory: /Users/mks/Predictive Analytics/Individual Assignment/Image Classification


In [11]:
# Create a Kaggle API key
# The file should initially be in the current directory where the command is executed.
# Moving key to proper location
import os

# Define the current and target paths
current_directory = os.getcwd()
kaggle_json_current = 'kaggle.json'  # Assuming the file is in the current directory
kaggle_json_target = os.path.join(current_directory, '.kaggle', 'kaggle.json')

try:
    # Check if kaggle.json is in the current directory
    if os.path.exists(kaggle_json_current):
        # Create the .kaggle directory in the current directory if it doesn't exist
        os.makedirs(os.path.dirname(kaggle_json_target), exist_ok=True)
        
        # Move the file to the new .kaggle directory
        os.rename(kaggle_json_current, kaggle_json_target)
        
        print(f"kaggle.json has been moved to {kaggle_json_target}.")
    
    elif os.path.exists(kaggle_json_target):
        print("kaggle.json already exists in the target location within the current directory.")

    else:
        print("kaggle.json not found in the current directory or the target location. Please ensure the file is available.")

except Exception as e:
    print(f"An error occurred: {e}")

kaggle.json has been moved to /Users/mks/Predictive Analytics/Individual Assignment/Image Classification/.kaggle/kaggle.json.


In [21]:
# Install Kaggle API
!pip install -q kaggle
!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
100%|███████████████████████████████████████▊| 345M/346M [00:08<00:00, 51.0MB/s]
100%|████████████████████████████████████████| 346M/346M [00:08<00:00, 43.9MB/s]


In [23]:
#unzip the files if they are not already unzipped

import os

target_dir = "intel_dataset"
zip_file = "intel-image-classification.zip"

if not os.path.isdir(target_dir):
    print(f"Directory '{target_dir}' not found. Unzipping dataset...")
    # Using the shell command with error handling
    try:
        get_ipython().system(f"unzip -q {zip_file} -d {target_dir}")
        print("Unzip complete.")
    except Exception as e:
        print(f"An error occurred while unzipping: {e}")
else:
    print(f"Directory '{target_dir}' already exists. Skipping unzip.")

Directory 'intel_dataset' not found. Unzipping dataset...
Unzip complete.


In [27]:
# For a check on whether each of the folder is downloaded
#Count of images in each folder
import os

def count_images_in_directory(directory):
    counts = {}
    # Loop over each subfolder (class) in the given directory
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            # Count common image files
            files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            counts[folder] = len(files)
    return counts

# Define paths to the seg_train, seg_test, and seg_pred directories
train_dir = os.path.join('intel_dataset', 'seg_train', 'seg_train')
test_dir = os.path.join('intel_dataset', 'seg_test', 'seg_test')
pred_dir = os.path.join('intel_dataset', 'seg_pred')

print("\nIn our directory, we current have these downloaded:")
print("\nTrain set counts:")
print(count_images_in_directory(train_dir))

print("\nTest set counts:")
print(count_images_in_directory(test_dir))

print("\nPrediction set counts:")
print(count_images_in_directory(pred_dir))

print("\nCross check these numbers against the Kaggle website datacard to ensure we have all images downloaded.")


In our directory, we current have these downloaded:

Train set counts:
{'forest': 2271, 'buildings': 2191, 'glacier': 2404, 'street': 2382, 'mountain': 2512, 'sea': 2274}

Test set counts:
{'forest': 474, 'buildings': 437, 'glacier': 553, 'street': 501, 'mountain': 525, 'sea': 510}

Prediction set counts:
{'seg_pred': 7301}

Cross check these numbers against the Kaggle website datacard to ensure we have all images downloaded.


In [29]:
#Creating a copy of the original dataset for our transformation and augmentation works
import os
import shutil

# Define the source and destination directories
src_dir = 'intel_dataset'
dest_dir = 'intel_dataset_copy'

# Only copy if the destination folder does not exist
if not os.path.exists(dest_dir):
    shutil.copytree(src_dir, dest_dir)
    print("Dataset copied to new folder for analysis and transformations.")
else:
    print("Copy folder already exists. Skipping dataset copy.")

Dataset copied to new folder for analysis and transformations.


In [12]:
# Code to delete the copy folder if needed.
# import os
# import shutil

# dest_dir = 'intel_dataset_copy'

# if os.path.exists(dest_dir):
#    shutil.rmtree(dest_dir)
#    print(f"Folder '{dest_dir}' has been deleted.")
# else:
#    print(f"Folder '{dest_dir}' does not exist.")

Folder 'intel_dataset_copy' does not exist.


Getting the summary statistics of the images from each of the classes from test and training data:

In [14]:
import os
import numpy as np
import statistics
from PIL import Image

# Define functions for computing resolution statistics
def image_resolution_stats(directory, sample_limit=50):
    stats = {}
    classes = sorted([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))])
    for cls in classes:
        cls_path = os.path.join(directory, cls)
        images = sorted([f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        sampled_images = images[:min(sample_limit, len(images))]
        widths = []
        heights = []
        for img_name in sampled_images:
            img_path = os.path.join(cls_path, img_name)
            try:
                with Image.open(img_path) as img:
                    w, h = img.size
                    widths.append(w)
                    heights.append(h)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
        if widths and heights:
            stats[cls] = {
                'width_min': min(widths),
                'width_max': max(widths),
                'width_mean': np.mean(widths),
                'width_median': statistics.median(widths),
                'width_std': np.std(widths),
                'height_min': min(heights),
                'height_max': max(heights),
                'height_mean': np.mean(heights),
                'height_median': statistics.median(heights),
                'height_std': np.std(heights)
            }
    return stats

def print_resolution_stats(stats, dataset_name="Dataset"):
    print(f"\n=== {dataset_name} Resolution Stats ===")
    for cls, s in stats.items():
        print(f"\nClass: {cls.capitalize()}")
        print(f"  Width  -> Min: {s['width_min']}, Max: {s['width_max']}, "
              f"Mean: {s['width_mean']:.2f}, Median: {s['width_median']}, Std: {s['width_std']:.2f}")
        print(f"  Height -> Min: {s['height_min']}, Max: {s['height_max']}, "
              f"Mean: {s['height_mean']:.2f}, Median: {s['height_median']}, Std: {s['height_std']:.2f}")

# Define functions for computing file size summary statistics
def file_size_stats(directory, sample_limit=50):
    stats = {}
    classes = sorted([d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))])
    for cls in classes:
        cls_path = os.path.join(directory, cls)
        images = sorted([f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        sampled_images = images[:min(sample_limit, len(images))]
        sizes = []
        for img_name in sampled_images:
            img_path = os.path.join(cls_path, img_name)
            try:
                size_kb = os.path.getsize(img_path) / 1024  # Convert bytes to kilobytes
                sizes.append(size_kb)
            except Exception as e:
                print(f"Error getting size for {img_path}: {e}")
        if sizes:
            stats[cls] = {
                'min': min(sizes),
                'max': max(sizes),
                'mean': np.mean(sizes),
                'median': statistics.median(sizes),
                'std': np.std(sizes)
            }
    return stats

def print_file_size_stats(stats, dataset_name="Dataset"):
    print(f"\n=== {dataset_name} File Size Stats (in KB) ===")
    for cls, s in stats.items():
        print(f"\nClass: {cls.capitalize()}")
        print(f"  Min: {s['min']:.2f} KB")
        print(f"  Max: {s['max']:.2f} KB")
        print(f"  Mean: {s['mean']:.2f} KB")
        print(f"  Median: {s['median']:.2f} KB")
        print(f"  Std: {s['std']:.2f} KB")

# Define new directories for the copied dataset
new_train_dir = os.path.join('intel_dataset_copy', 'seg_train', 'seg_train')
new_test_dir  = os.path.join('intel_dataset_copy', 'seg_test', 'seg_test')

# Compute resolution stats on the new folders
new_train_resolution_stats = image_resolution_stats(new_train_dir)
new_test_resolution_stats  = image_resolution_stats(new_test_dir)

# Print resolution stats
print_resolution_stats(new_train_resolution_stats, "New Training Set")
print_resolution_stats(new_test_resolution_stats, "New Test Set")

# Compute file size stats on the new folders
new_train_file_stats = file_size_stats(new_train_dir)
new_test_file_stats  = file_size_stats(new_test_dir)

# Print file size stats
print_file_size_stats(new_train_file_stats, "New Training Set")
print_file_size_stats(new_test_file_stats, "New Test Set")


=== New Training Set Resolution Stats ===

Class: Buildings
  Width  -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00
  Height -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00

Class: Forest
  Width  -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00
  Height -> Min: 113, Max: 150, Mean: 149.26, Median: 150.0, Std: 5.18

Class: Glacier
  Width  -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00
  Height -> Min: 110, Max: 150, Mean: 149.20, Median: 150.0, Std: 5.60

Class: Mountain
  Width  -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00
  Height -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00

Class: Sea
  Width  -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00
  Height -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00

Class: Street
  Width  -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00
  Height -> Min: 150, Max: 150, Mean: 150.00, Median: 150.0, Std: 0.00

=== New Tes